In [2]:
# TODO in class
# install BeautifulSoup if you do not have it already
# pip install beautifulsoup4 
# note the 4 at the end
# find two URLs that you want to scrape on SS.com
# scrape the data from the first page
# scrape the data from the second page
# you can use the functions I provided in the class

# save both as csv files
# and make plot of the data - up to you what you want to plot

In [3]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import time

In [4]:
url = "https://www.ss.com/en/animals/cats/"

In [33]:
req = requests.get(url)
req.status_code

200

In [9]:
soup = BeautifulSoup(req.text, 'lxml')
soup.title

In [10]:
tables = soup.find_all("table")
headline = soup.find("tr", {"id":"head_line"})
headtds = headline.find_all("td")
headline.text
headtds[0].text

'\n\xa0Advertisements\r\n\n\n\ndate\n'

In [12]:
headcolumns = [el.text for el in headtds[1:]]
headcolumns

['Breed', 'Age', 'Price']

In [15]:
def getColList(soup):
    column_list = ["description","url"] # we decided to that we need these two column names no matter the html
    headline = soup.find("tr", {"id":"head_line"})
    headtds = headline.find_all("td")
    headcolumns = [el.text for el in headtds[1:]] # this will get all column names starting with 2nd in HTML
    column_list += headcolumns
    return column_list

In [18]:
column_names = getColList(soup)
column_names

['description', 'url', 'Breed', 'Age', 'Price']

In [19]:
trows = soup.find_all('tr')

In [20]:
rows = [row for row in trows if row.get('id',"").startswith("tr_") and not row.get('id',"").startswith("tr_bnr") ]

In [21]:
def getRowList(soup):
    trows = soup.find_all('tr')
    aprows = [row for row in trows if row.get('id',"").startswith("tr_") and not row.get('id',"").startswith("tr_bnr") ]
    return aprows

In [22]:
rows = getRowList(soup)

In [23]:
first_ad = rows[0]

In [35]:
row_tds = first_ad.find_all('td')
row_tds[:2]

[<td class="msga2 pp0"><input id="c51617380" name="mid[]" type="checkbox" value="51617380_75037_0"/></td>,
 <td class="msga2"><a href="/msg/en/animals/cats/scottish-fold/bpdexo.html" id="im51617380"><img alt="" class="isfoto foto_list" src="https://i.ss.com/gallery/5/975/243596/48719066.th2.jpg"/></a></td>]

In [26]:
def getRow(row,colist=column_names):
    row_tds = row.find_all('td')
    rowDict = {}
    if len(row_tds) < 3: # a little sanity check
        print("Hmm bad row")
        return rowDict
    
    rowDict[colist[0]] = row_tds[2].text # so the big assumption is that we always get description in 3rd column
    rowDict[colist[1]] = "https://ss.com" + row_tds[1].find('a').get('href')
    for td,key in zip(row_tds[3:],colist[2:]): 
        rowDict[key] = td.text
    return rowDict

In [27]:
getRow(rows[6])

{'description': 'Šis strīpainis skaistulis ir nopietns kaķu kungs. \r\nBet neskatoties uz visu nopietnību, Jelgava and reg.',
 'url': 'https://ss.com/msg/en/animals/cats/outbred/jhmck.html',
 'Breed': 'Outbred',
 'Age': '5 years',
 'Price': '70  €'}

In [28]:
def getRows(rowlist,colist=column_names):
    return [getRow(row, colist=colist) for row in rowlist]

In [29]:
def getDFfromURL(url):
    
    req = requests.get(url)
    if req.status_code != 200:
        print("Request Fail with", req.status_code)
        return None 
    soup = BeautifulSoup(req.text, 'lxml')
    column_names = getColList(soup)
    rowlist = getRowList(soup)
    rows = getRows(rowlist,colist=column_names)
    return pd.DataFrame(rows, columns=column_names)

In [30]:
cats = url
idf = getDFfromURL(cats)
idf.head()

,description,url,Breed,Age,Price
0,"Отдаю за пожертвование идеального котика, кото...",https://ss.com/msg/en/animals/cats/unknown-bre...,Unknown breed,5 years,-
1,Pārdodu divus kaķu trauciņusRiga,https://ss.com/msg/en/animals/cats/accessories...,-,-,20 €
2,Отдам котят в хорошие руки.Daugavpils and reg.,https://ss.com/msg/en/animals/cats/outbred/cxp...,Outbred,1 mon.,-
3,Piedāvājam tīršķirnes Skotu fold meitenīti ar ...,https://ss.com/msg/en/animals/cats/scottish-fo...,Scottish fold,2 mon.,500 €
4,Balti-melna kaķenīte ir mīļuma iemiesojums. \r...,https://ss.com/msg/en/animals/cats/outbred/jgh...,Outbred,2 years,70 €


In [31]:
idf.to_csv("cats.csv")

In [32]:
cats = pd.read_csv(r"C:\Users\dovil\Desktop\she goes tech\SGT\cats.csv")
cats

,Unnamed: 0,description,url,Breed,Age,Price
0,0,"Отдаю за пожертвование идеального котика, кото...",https://ss.com/msg/en/animals/cats/unknown-bre...,Unknown breed,5 years,-
1,1,Pārdodu divus kaķu trauciņusRiga,https://ss.com/msg/en/animals/cats/accessories...,-,-,20 €
2,2,Отдам котят в хорошие руки.Daugavpils and reg.,https://ss.com/msg/en/animals/cats/outbred/cxp...,Outbred,1 mon.,-
3,3,Piedāvājam tīršķirnes Skotu fold meitenīti ar ...,https://ss.com/msg/en/animals/cats/scottish-fo...,Scottish fold,2 mon.,500 €
4,4,Balti-melna kaķenīte ir mīļuma iemiesojums. \r...,https://ss.com/msg/en/animals/cats/outbred/jgh...,Outbred,2 years,70 €
5,5,Вот такой красавчик рыжий с розовым носиком до...,https://ss.com/msg/en/animals/cats/outbred/odn...,Outbred,1 years,70 €
6,6,"”Es tik ļoti ilgojos pēc cilvēka mīlestības, t...",https://ss.com/msg/en/animals/cats/outbred/jbb...,Outbred,5 years,70 €
7,7,Rudulis ir īsts skaistulis un mīļuma iemiesoju...,https://ss.com/msg/en/animals/cats/outbred/kxn...,Outbred,4 years,70 €
8,8,Kuplā kažociņa īpašniece meklē savas mājas. \r...,https://ss.com/msg/en/animals/cats/outbred/hff...,Outbred,1.5 years,70 €
9,9,Šis strīpainis skaistulis ir nopietns kaķu kun...,https://ss.com/msg/en/animals/cats/outbred/jhm...,Outbred,5 years,70 €
